In [348]:
%%writefile ../../src/features/compare_tide.py
from scipy.spatial import KDTree
import ttide as tt
import datetime
import numpy as np
import scipy.io as sio

def read_atg(atg_data,site_id,constit_list):

    site_data = {}
    for key in ['site_id','name','lat','lon','amp','Gphase','reclen','delta_t','meas_type','ref']:
        site_data[key] =np.squeeze(atg_data[key][0,0][site_id-1])
    site_data['constit']=np.squeeze(atg_data['constit'][0,0][:])
    site_data['name'] = site_data['name'].strip()

    cm2m = 1.0/100.0
    for const in constit_list:

        atg_con_ind = list(site_data['constit']).index(const)
        site_data[const]=np.array([site_data['amp'][atg_con_ind]*cm2m, site_data['Gphase'][atg_con_ind]])
        
    return site_data

def station_ttide(zeta_da,roms_mask_da,lat_t,lon_t,stime,constit_list):
    
    zeta_flat = zeta_da.stack(etaxi = ('eta_rho','xi_rho'))
    mask_flat = roms_mask_da.stack(etaxi = ('eta_rho','xi_rho'))
    
    lat_s = zeta_flat.lat_rho.values[mask_flat.values==True]
    lon_s = zeta_flat.lon_rho.values[mask_flat.values==True]
    zeta_s = zeta_flat.values[:,mask_flat.values==True]
    
    points = np.column_stack((lat_s,lon_s))
    tree = KDTree(points)
    
    target = np.column_stack((lat_t,lon_t))
    dist, ind = tree.query(target)
    
    tmp={}
    tmp['roms_signal'] = zeta_s[:,ind].squeeze()
    tmp['roms_ind'],tmp['dist_to ATG'] = ind,dist
    lat_r = lat_s[ind]
    lon_r = lon_s[ind]
    
    print('atg lat(lon): %.2f,%.2f'%(lat_t,lon_t))
    print('roms lat(lon): %.2f,%.2f'%(lat_r,lon_r))
    tmp['t_tide']=tt.t_tide(tmp['roms_signal'],dt=1,stime=stime,lat=lat_r,out_style=None)


    for const in constit_list:
        tide_con_ind = list(tmp['t_tide']['nameu']).index(str.encode(const+'  ')) 
        tmp[const]=tmp['t_tide']['tidecon'][tide_con_ind]
        
    return tmp

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def complex_rmse(predictions, targets):
    return np.sqrt(0.5*(((predictions - targets)*np.conjugate(predictions - targets)).real.mean()))

def calc_rmse(station_dict,constit_list):
    
    const_rmse={}

    for constit in constit_list:
        
        tt_amp_all = []
        atg_amp_all = []

        tt_phi_all =[]
        atg_phi_all =[]

        tt_z_all = []
        atg_z_all = []

        for station,data in station_dict.items():
            
            
            tt_amp = data['tt'][constit][0]
            atg_amp = data['atg'][constit][0]

            tt_phi = data['tt'][constit][2]
            atg_phi = data['atg'][constit][1]

            tt_amp_all.append(tt_amp)
            atg_amp_all.append(atg_amp)

            tt_phi_all.append(tt_phi)
            atg_phi_all.append(atg_phi)

            tt_z_all.append(tt_amp * np.exp(1j*tt_phi))
            atg_z_all.append(atg_amp * np.exp(1j*atg_phi))

        const_rmse[constit] = {}
        
        const_rmse[constit]['amp']=rmse(np.asarray(tt_amp_all),np.asarray(atg_amp_all))
        const_rmse[constit]['phase']=rmse(np.asarray(tt_phi_all),np.asarray(atg_phi_all))
        const_rmse[constit]['complex_amp']=complex_rmse(np.asarray(atg_z_all),np.asarray(tt_z_all))
        
    return const_rmse 

def print_station_dict(station_dict,constit_list):
    print("Amp(amp_err)[m]:  atg      roms    ||   phase(phase_err)[deg]:  atg       roms")
    for constit in constit_list:
        print(constit)
        for station_id,data in station_dict.items():  
            print('Station ',station_id,\
                  "      %0.2f"%data['atg'][constit][0], "  %0.2f(%0.2f)"%(data['tt'][constit][0],data['tt'][constit][1]),\
                  "                          %0.2f"%data['atg'][constit][1], "  %0.2f(%0.2f)"%(data['tt'][constit][2],data['tt'][constit][3])) 

def print_rmse(rmse_dict,constit_list):
    print('RMSE:  amp [m]    phase [deg]   complex[m]')
    for constit,data in rmse_dict.items():
        print(constit,'       %.2f        %.2f         %.2f '%(data['amp'],data['phase'],data['complex_amp']))

def compare_tide(atg_mat_path,roms_zeta_da,roms_mask_da,stime=datetime.datetime(2007,1,1),constit_list = ['M2','O1'],station_list= [24,30]):

    print('stime = ',stime,' constits = ',constit_list,'stations = ',station_list)
    mat_content = sio.loadmat(atg_mat_path)
    atg_data = mat_content['atg']
    
    station_dict = {}
    
    for station in station_list:
        
        print('processing station ',station)
        station_dict[station] = {}

        atg_dict = read_atg(atg_data,station,constit_list)
        lat = atg_dict['lat']
        lon = atg_dict['lon']
        tt_dict = station_ttide(roms_zeta_da,roms_mask_da,lat,lon,stime,constit_list)

        #print_comparison(tt_dict,atg_dict,constit_list)
        
        station_dict[station]['atg'] = atg_dict
        station_dict[station]['tt'] = tt_dict
        
    print_station_dict(station_dict,constit_list)
    
   
        
    rmse_dict = calc_rmse(station_dict,constit_list)
    
    print_rmse(rmse_dict,constit_list)
    
    
        
    return station_dict,rmse_dict

Overwriting ../../src/features/compare_tide.py


In [29]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

src_dir = os.environ.get('srcdir')
print(src_dir)

sys.path.append(src_dir)

# always reload modules marked with "%aimport"
%load_ext autoreload
%autoreload 1

%aimport features.compare_tide
import features.compare_tide as compare_tide

/home/ubuntu/bigStick/tidal_melting/src
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [346]:
import xarray as xr
import datetime

atg_path = os.path.join(os.environ.get('projdir'),'data','analysis','external','atg','ATG_ocean_height_2010_0908.mat')
#his = os.path.join(os.environ.get('rawdir'),'waom10_Cd','ocean_his_x10Cd_0006.nc')
his = os.path.join(os.environ.get('rawdir'),'waom10_tidal_mask','ocean_his_tm_smooth_0006.nc')
rds = xr.open_dataset(his)
zeta = rds.zeta.sel(ocean_time=slice('2000-1-14','2000-1-28'))
#mask = xr.open_dataset(his).mask_rho
#zeta.summary()
#zeta = xr.open_dataset(his).zeta

station_id = [1,3,15]
const_list = ['M2']
stime=datetime.datetime(2007,1,1)

comp,RMSE = compare_tide(atg_path,zeta,rds.mask_rho,station_list=station_id)

processing station  1
atg lat(lon): -65.25,-64.27
roms lat(lon): -65.23,-64.19
processing station  3
atg lat(lon): -67.57,-68.13
roms lat(lon): -67.65,-68.03
processing station  15
atg lat(lon): -61.47,-61.28
roms lat(lon): -61.50,-61.24
Amp(amp_err)[m]:  atg      roms    ||   phase(phase_err)[deg]:  atg       roms
M2
Station  1       0.19   0.60(0.31)                           287.87   196.51(38.38)
Station  3       0.12   0.59(0.41)                           265.51   212.58(38.73)
Station  15       0.32   0.55(0.33)                           268.00   156.18(34.44)
O1
Station  1       0.29   0.21(0.02)                           69.77   185.54(4.54)
Station  3       0.24   0.17(0.02)                           77.01   182.15(6.31)
Station  15       0.22   0.12(0.01)                           51.00   136.30(7.45)
RMSE:  amp [m]    phase [deg]   complex[m]
M2        0.39        88.79         0.48 
O1        0.09        102.85         0.27 
